<h2 style="text-align:center;">ReAct Agent</h2>

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Zoomable Image</title>
    <style>
        body {
            margin: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            background-color: #f0f0f0;
        }
        #graph {
            width: 60%;
            height: auto;
        }
    </style>
    <script src="https://unpkg.com/@panzoom/panzoom@9.4.0/dist/panzoom.min.js"></script>
</head>
<body>
    <div id="graph-container">
        <center><img id="graph" src="./data/react.png" alt="Language Graph"></center>
    </div>
    <script>
        const element = document.getElementById('graph');
        const panzoom = Panzoom(element, {
            maxScale: 3,
            minScale: 0.5
        });
        element.parentElement.addEventListener('wheel', panzoom.zoomWithWheel);
    </script>
</body>
</html>


#### Install Libraries

In [ ]:
# !pip install -U langgraph
# !pip install langchain-openai
# !pip install wikipedia

#### Instantiate LLM

In [ ]:
from langchain_openai import ChatOpenAI
import os, getpass

In [ ]:
# if not os.environ.get("OPENAI_API_KEY"):
    # os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

llm = ChatOpenAI(temperature=0, model="gpt-4.1-nano", api_key=os.environ["OPENAI_API_KEY"]) # type: ignore
# llm.invoke("Hi")

In [ ]:
llm.invoke("Hi")

#### Initialize langsmith

In [ ]:
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "React Agent with LangGraph"

#### State for the Graph

In [ ]:
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage

In [ ]:
human_message = HumanMessage(content=f"Who won the ODI CWC 2011?", name="theailearner")
human_message

In [ ]:
human_message.pretty_print()

In [ ]:
state = MessagesState({"messages" : [human_message]}) # type: ignore
state

In [ ]:
res = llm.invoke(state["messages"])
res

In [ ]:
state['messages'].append(res) # type: ignore
state

In [ ]:
state['messages'].append(HumanMessage(content=f"Which team did india beat in finals?", name="theailearner")) # type: ignore

In [ ]:
for m in state["messages"]:
    m.pretty_print()

In [ ]:
res = llm.invoke(state["messages"])
print(res.content)

#### Tool

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1)) # type: ignore

In [ ]:
res = wikipedia.run("fifa world cup 2026")
print(res)

In [ ]:
def search_wikipedia(query: str) -> str:
    '''Searches Wikipedia for the given query and returns a summary of the results.
    Args:
        query (str): The search query.'''
    
    return wikipedia.run(query)

In [ ]:
llm_with_tools = llm.bind_tools([search_wikipedia])

#### Graph

In [ ]:
def call_llm(state : MessagesState) -> dict:
    response = llm_with_tools.invoke(state["messages"])
    return {"messages" : response}

In [ ]:
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph import StateGraph, START
from IPython.display import Image, display # type: ignore

In [ ]:
builder = StateGraph(MessagesState)
builder.add_node("call_llm", call_llm)
builder.add_node("tools", ToolNode([search_wikipedia]))
builder.add_edge(START, "call_llm")
builder.add_conditional_edges("call_llm", tools_condition)
builder.add_edge("tools", "call_llm")
react_graph = builder.compile()

display(Image(react_graph.get_graph().draw_mermaid_png()))

In [ ]:
msg = [HumanMessage(content="Where will the FIFA World Cup 2026 be held?")]

In [ ]:
results = react_graph.invoke({"messages": msg}) # type: ignore

In [ ]:
for m in results["messages"]:
    print(m.pretty_print())

In [ ]:
results["messages"][-1].content

#### with prebuilt component

In [ ]:
from langgraph.prebuilt import create_react_agent

In [ ]:
prebuilt_agent = create_react_agent(model = llm, tools = [search_wikipedia])

In [ ]:
display(Image(prebuilt_agent.get_graph().draw_mermaid_png()))

In [ ]:
response = prebuilt_agent.invoke({"messages": msg})

In [ ]:
for m in response['messages']:
    m.pretty_print()

In [ ]:
response['messages'][-1].pretty_print()